In [5]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime
from datetime import timedelta

In [6]:
from couchbase.exceptions import CouchbaseException
from couchbase.auth import PasswordAuthenticator
from couchbase.cluster import Cluster
from couchbase.options import ClusterOptions

In [7]:
# Configuración de Couchbase
endpoint = "couchbases://cb.pps-dendznwwlsxh.cloud.couchbase.com"
username = "BDD_PULSOSPOLITICOS"
password = "Administrador32#"  # Reemplazar con la contraseña real
bucket_name = "BDD_PULSOSPOLITICOS"
scope_name = "OpinionPublicaPolitica"
collection_name = "NoticiasPoliticas"

In [8]:
# Couchbase connection setup
auth = PasswordAuthenticator(username, password)
options = ClusterOptions(auth)
options.apply_profile("wan_development")

cluster = Cluster(endpoint, options)
cluster.wait_until_ready(timedelta(seconds=5))
cb = cluster.bucket(bucket_name)
cb_coll = cb.scope(scope_name).collection(collection_name)

In [51]:
base_url = "https://cnnespanol.cnn.com/latinoamerica/ecuador/"

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
response = requests.get(base_url, headers=headers)


In [58]:
# URL de la página de noticias políticas en El Telégrafo
url = "https://www.eltelegrafo.com.ec/politica"

# Obtener la respuesta del servidor
response = requests.get(url)

# Si la respuesta es exitosa, analizamos el contenido con BeautifulSoup
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')


In [77]:
# URL de la página principal de Teleamazonas
url = 'https://www.teleamazonas.com/'

# Lista para almacenar las noticias
noticias = []

# Hacer la solicitud GET para obtener el contenido de la página
response = requests.get(url)

if response.status_code == 200:
    print(f"Acceso exitoso a la página principal ({url})")
    
    # Crear el objeto BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Buscar todos los títulos de noticias con la clase 'entry-title'
    articulos = soup.find_all('h3', class_='entry-title')
    
    # Extraer el título y el enlace de cada artículo
    for articulo in articulos:
        titulo = articulo.get_text(strip=True)  # Extraer el texto del título
        enlace = articulo.find('a')['href'] if articulo.find('a') else None  # Extraer el enlace
        
        if enlace:
            noticias.append({
                'titulo': titulo,
                'enlace': enlace
            })
else:
    print(f"Error al acceder a la página principal. Código de estado: {response.status_code}")

# Convertir las noticias extraídas en un DataFrame de pandas
df_noticias = pd.DataFrame(noticias)

# Guardar el DataFrame en un archivo JSON
df_noticias.to_json('noticias_teleamazonas_principal.json', orient='records', lines=True, force_ascii=False)

print("Datos guardados en 'noticias_teleamazonas_principal.json'")


Acceso exitoso a la página principal (https://www.teleamazonas.com/)
Datos guardados en 'noticias_teleamazonas_principal.json'


In [78]:
# URL de la página principal de El Universo
url = 'https://www.eluniverso.com/'

# Lista para almacenar las noticias
noticias = []

# Hacer la solicitud GET para obtener el contenido de la página
response = requests.get(url)

if response.status_code == 200:
    print(f"Acceso exitoso a la página principal ({url})")
    
    # Crear el objeto BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Buscar todos los títulos de noticias con la clase 'text-base m-0 font-bold font-primary'
    articulos = soup.find_all('h2', class_='text-base m-0 font-bold font-primary')
    
    # Extraer el título y el enlace de cada artículo
    for articulo in articulos:
        titulo = articulo.get_text(strip=True)  # Extraer el texto del título
        enlace = articulo.find('a')['href'] if articulo.find('a') else None  # Extraer el enlace
        
        if enlace:
            noticias.append({
                'titulo': titulo,
                'enlace': 'https://www.eluniverso.com' + enlace  # Concatenar la URL base
            })
else:
    print(f"Error al acceder a la página principal. Código de estado: {response.status_code}")

# Convertir las noticias extraídas en un DataFrame de pandas
df_noticias = pd.DataFrame(noticias)

# Guardar el DataFrame en un archivo JSON
df_noticias.to_json('noticias_eluniverso_principal.json', orient='records', lines=True, force_ascii=False)

print("Datos guardados en 'noticias_eluniverso_principal.json'")

Acceso exitoso a la página principal (https://www.eluniverso.com/)
Datos guardados en 'noticias_eluniverso_principal.json'


In [59]:
# Crear una lista para almacenar las noticias
noticias = []

# Buscar todos los artículos de noticias en el HTML
for item in soup.find_all('div', class_='catItemView groupLeading'):
    titulo_element = item.find('h1', class_='story-heading')
    if titulo_element:
        titulo = titulo_element.get_text(strip=True)
        enlace = titulo_element.find('a').get('href')
        if enlace and enlace.startswith('/'):
            enlace = f"https://www.eltelegrafo.com.ec{enlace}"
        noticias.append({'titulo': titulo, 'enlace': enlace})


In [54]:
# Verificar que la solicitud fue exitosa
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Buscar los artículos de noticias con la clase 'container__headline-text'
    noticias = soup.find_all('span', class_='container__headline-text')
    
    # Listas para almacenar los datos
    titulos = []
    enlaces = []
    
    # Recorrer las noticias y extraer los títulos y los enlaces
    for noticia in noticias:
        # Buscar el enlace en el elemento contenedor
        enlace = noticia.find_parent('a')['href']
        
        # Formar la URL completa en caso de que el enlace sea relativo
        if enlace.startswith('/'):
            enlace = 'https://cnnespanol.cnn.com' + enlace
        
        # Agregar el título y el enlace a las listas
        titulos.append(noticia.get_text())
        enlaces.append(enlace)
    
    # Crear un DataFrame de Pandas
    df = pd.DataFrame({
        'titulo': titulos,
        'link': enlaces
    })
    
    # Guardar los resultados en un archivo JSON
    df.to_json('noticias_ecuador.json', orient='records', force_ascii=False, lines=True)
    
    print("Datos guardados en 'noticias_ecuador.json'")
else:
    print("Error al acceder a la página:", response.status_code)

Datos guardados en 'noticias_ecuador.json'


In [ ]:
# URL de la página principal de Expreso
url = 'https://www.expreso.ec'

# Realizar la solicitud GET
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Crear una lista para almacenar las noticias
noticias = []

# Buscar los títulos y enlaces de las noticias dentro de <h2 class="c-article__title">
articulos = soup.find_all('h2', class_='c-article__title')

# Iterar sobre los artículos encontrados y extraer el título y el enlace
for articulo in articulos:
    titulo = articulo.get_text(strip=True)
    enlace = articulo.find('a')['href'] if articulo.find('a') else None
    if enlace:  # Verifica si el enlace no es None
        noticias.append({
            'titulo': titulo,
            'enlace': url + enlace  # Agrega la URL base al enlace relativo
        })

# Convertir la lista de noticias en un DataFrame de pandas
df_noticias = pd.DataFrame(noticias)

# Guardar los datos en un archivo JSON
df_noticias.to_json('noticias_expreso.json', orient='records', lines=True, force_ascii=False)

# Imprimir el DataFrame
print(df_noticias)

print("Datos guardados en 'noticias_expreso.json'")

In [ ]:
# URL de la página principal de Expreso
url = 'https://www.expreso.ec'

# Realizar la solicitud GET
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Crear una lista para almacenar las noticias
noticias = []

# Buscar los títulos y enlaces de las noticias dentro de <h2 class="c-article__title">
articulos = soup.find_all('h2', class_='c-article__title')

# Iterar sobre los artículos encontrados y extraer el título y el enlace
for articulo in articulos:
    titulo = articulo.get_text(strip=True)
    enlace = articulo.find('a')['href'] if articulo.find('a') else None
    if enlace:  # Verifica si el enlace no es None
        noticias.append({
            'titulo': titulo,
            'enlace': url + enlace  # Agrega la URL base al enlace relativo
        })

# Convertir la lista de noticias en un DataFrame de pandas
df_noticias = pd.DataFrame(noticias)

# Guardar los datos en un archivo JSON
df_noticias.to_json('noticias_expreso.json', orient='records', lines=True, force_ascii=False)

# Imprimir el DataFrame
print(df_noticias)

print("Datos guardados en 'noticias_expreso.json'")

In [82]:
# URL de Radio Sucre
url = "https://radiosucre.com.ec"

# Realizar la solicitud HTTP
response = requests.get(url)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    print("Acceso exitoso a la página.")
    
    # Analizar el contenido con BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Encontrar los títulos de las noticias
    articulos = soup.find_all('h4', class_='title')
    
    # Lista para almacenar las noticias
    noticias = []
    
    # Extraer los títulos y enlaces
    for articulo in articulos:
        titulo = articulo.get_text(strip=True)
        enlace = articulo.find('a')['href']
        
        noticias.append({
            'titulo': titulo,
            'enlace': enlace
        })
    
    # Convertir a DataFrame de Pandas
    df = pd.DataFrame(noticias)
    
    # Guardar como archivo JSON
    df.to_json('noticias_radio_sucre.json', orient='records', lines=True, force_ascii=False)
    
    print("Datos guardados en 'noticias_radio_sucre.json'")
else:
    print(f"Error al acceder a la página. Código de estado: {response.status_code}")

Acceso exitoso a la página.
Datos guardados en 'noticias_radio_sucre.json'


In [83]:
# URL de Notimundo
url = "https://notimundo.com.ec"

# Realizar la solicitud HTTP
response = requests.get(url)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    print("Acceso exitoso a la página.")
    
    # Analizar el contenido con BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Encontrar los títulos de las noticias
    articulos = soup.find_all('h3', class_='entry-title td-module-title')
    
    # Lista para almacenar las noticias
    noticias = []
    
    # Extraer los títulos y enlaces
    for articulo in articulos:
        titulo = articulo.get_text(strip=True)
        enlace = articulo.find('a')['href']
        
        noticias.append({
            'titulo': titulo,
            'enlace': enlace
        })
    
    # Convertir a DataFrame de Pandas
    df = pd.DataFrame(noticias)
    
    # Guardar como archivo JSON
    df.to_json('noticias_notimundo.json', orient='records', lines=True, force_ascii=False)
    
    print("Datos guardados en 'noticias_notimundo.json'")
else:
    print(f"Error al acceder a la página. Código de estado: {response.status_code}")

Acceso exitoso a la página.
Datos guardados en 'noticias_notimundo.json'


In [84]:
# URL de la página de Vistazo
url = 'https://www.vistazo.com/'

# Realizar la solicitud HTTP
response = requests.get(url)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    print(f"Accediendo a la página: {url}")
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Buscar los títulos de las noticias dentro de las etiquetas <h2>
    articulos = soup.find_all('h2')
    
    # Crear una lista para almacenar las noticias
    noticias = []

    # Extraer título y enlace de cada noticia
    for articulo in articulos:
        titulo = articulo.get_text(strip=True)
        enlace = articulo.find_parent('a')['href'] if articulo.find_parent('a') else None
        
        # Añadir la noticia a la lista
        if enlace:
            noticias.append({
                'titulo': titulo,
                'enlace': 'https://www.vistazo.com' + enlace  # Añadir el dominio al enlace
            })
    
    # Convertir los datos a un DataFrame de pandas
    df = pd.DataFrame(noticias)

    # Guardar los datos en un archivo JSON
    df.to_json('noticias_vistazo.json', orient='records', lines=True, force_ascii=False)

    print(f"Datos guardados en 'noticias_vistazo.json'")

else:
    print(f"Error al acceder a la página. Código de estado: {response.status_code}")

Accediendo a la página: https://www.vistazo.com/
Datos guardados en 'noticias_vistazo.json'


In [85]:
# URL de la página de Extra
url = 'https://www.extra.ec/'

# Realizar la solicitud HTTP
response = requests.get(url)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    print(f"Accediendo a la página: {url}")
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Buscar los títulos de las noticias dentro de las etiquetas <h2> con la clase 'c-article__title'
    articulos = soup.find_all('h2', class_='c-article__title')
    
    # Crear una lista para almacenar las noticias
    noticias = []

    # Extraer título y enlace de cada noticia
    for articulo in articulos:
        titulo = articulo.get_text(strip=True)
        enlace = articulo.find('a')['href'] if articulo.find('a') else None
        
        # Añadir la noticia a la lista
        if enlace:
            noticias.append({
                'titulo': titulo,
                'enlace': 'https://www.extra.ec' + enlace  # Añadir el dominio al enlace
            })
    
    # Convertir los datos a un DataFrame de pandas
    df = pd.DataFrame(noticias)

    # Guardar los datos en un archivo JSON
    df.to_json('noticias_extra.json', orient='records', lines=True, force_ascii=False)

    print(f"Datos guardados en 'noticias_extra.json'")

else:
    print(f"Error al acceder a la página. Código de estado: {response.status_code}")

Accediendo a la página: https://www.extra.ec/
Datos guardados en 'noticias_extra.json'


In [86]:
# URL de la página de Hoy
url = 'https://www.hoy.com.ec/'

# Realizar la solicitud HTTP
response = requests.get(url)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    print(f"Accediendo a la página: {url}")
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Buscar los títulos de las noticias dentro de las etiquetas <h1> con la clase 'blog-entry-title entry-title'
    articulos = soup.find_all('h1', class_='blog-entry-title entry-title')
    
    # Crear una lista para almacenar las noticias
    noticias = []

    # Extraer título y enlace de cada noticia
    for articulo in articulos:
        titulo = articulo.get_text(strip=True)
        enlace = articulo.find('a')['href'] if articulo.find('a') else None
        
        # Añadir la noticia a la lista
        if enlace:
            noticias.append({
                'titulo': titulo,
                'enlace': 'https://www.hoy.com.ec' + enlace  # Añadir el dominio al enlace
            })
    
    # Convertir los datos a un DataFrame de pandas
    df = pd.DataFrame(noticias)

    # Guardar los datos en un archivo JSON
    df.to_json('noticias_hoy.json', orient='records', lines=True, force_ascii=False)

    print(f"Datos guardados en 'noticias_hoy.json'")

else:
    print(f"Error al acceder a la página. Código de estado: {response.status_code}")

Accediendo a la página: https://www.hoy.com.ec/
Datos guardados en 'noticias_hoy.json'


In [87]:
# URL de la página de La República
url = 'https://www.larepublica.ec/'

# Realizar la solicitud HTTP
response = requests.get(url)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    print(f"Accediendo a la página: {url}")
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Buscar los títulos de las noticias dentro de las etiquetas <h3> con la clase 'mh-custom-posts-xl-title'
    articulos = soup.find_all('h3', class_='mh-custom-posts-xl-title')
    
    # Crear una lista para almacenar las noticias
    noticias = []

    # Extraer título y enlace de cada noticia
    for articulo in articulos:
        titulo = articulo.get_text(strip=True)
        enlace = articulo.find('a')['href'] if articulo.find('a') else None
        
        # Añadir la noticia a la lista
        if enlace:
            noticias.append({
                'titulo': titulo,
                'enlace': enlace  # El enlace ya está completo
            })
    
    # Convertir los datos a un DataFrame de pandas
    df = pd.DataFrame(noticias)

    # Guardar los datos en un archivo JSON
    df.to_json('noticias_la_republica.json', orient='records', lines=True, force_ascii=False)

    print(f"Datos guardados en 'noticias_la_republica.json'")

else:
    print(f"Error al acceder a la página. Código de estado: {response.status_code}")

Accediendo a la página: https://www.larepublica.ec/
Datos guardados en 'noticias_la_republica.json'


In [89]:
# URL de la página de PlanV
url = 'https://www.planv.com.ec'

# Realizar la solicitud HTTP
response = requests.get(url)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    print(f"Accediendo a la página: {url}")
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Buscar los títulos de las noticias dentro de las etiquetas <h3> con la clase 'entry-title td-module-title'
    articulos = soup.find_all('h3', class_='entry-title td-module-title')
    
    # Crear una lista para almacenar las noticias
    noticias = []

    # Extraer título y enlace de cada noticia
    for articulo in articulos:
        titulo = articulo.get_text(strip=True)
        enlace = articulo.find('a')['href'] if articulo.find('a') else None
        
        # Añadir la noticia a la lista
        if enlace:
            noticias.append({
                'titulo': titulo,
                'enlace': enlace  # El enlace ya está completo
            })
    
    # Convertir los datos a un DataFrame de pandas
    df = pd.DataFrame(noticias)

    # Guardar los datos en un archivo JSON
    df.to_json('noticias_planv.json', orient='records', lines=True, force_ascii=False)

    print(f"Datos guardados en 'noticias_planv.json'")

else:
    print(f"Error al acceder a la página. Código de estado: {response.status_code}")

Accediendo a la página: https://www.planv.com.ec
Datos guardados en 'noticias_planv.json'


In [94]:
# URL de la página de Ecuador Times
url = 'https://www.ecuadortimes.net/'

# Encabezados para simular un navegador real
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Realizar la solicitud HTTP
response = requests.get(url, headers=headers)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    print(f"Accediendo a la página: {url}")
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Buscar los títulos de las noticias dentro de las etiquetas <h2> con la clase 'blog-title'
    articulos = soup.find_all('h2', class_='blog-title')
    
    # Crear una lista para almacenar las noticias
    noticias = []

    # Extraer título y enlace de cada noticia
    for articulo in articulos:
        enlace = articulo.find('a')  # Buscar el enlace dentro del h2
        if enlace:
            titulo = enlace.get_text(strip=True)
            url_noticia = enlace['href']  # Obtener el atributo href del enlace
            
            # Añadir la noticia a la lista
            noticias.append({
                'titulo': titulo,
                'enlace': url_noticia
            })
    
    # Convertir los datos a un DataFrame de pandas
    df = pd.DataFrame(noticias)

    # Guardar los datos en un archivo JSON
    df.to_json('noticias_ecuador_times.json', orient='records', lines=True, force_ascii=False)

    print(f"Datos guardados en 'noticias_ecuador_times.json'")

else:
    print(f"Error al acceder a la página. Código de estado: {response.status_code}")

Accediendo a la página: https://www.ecuadortimes.net/
Datos guardados en 'noticias_ecuador_times.json'


In [95]:
# URL de la página de BBC News
url = 'https://www.bbc.com/news'

# Encabezados para evitar restricciones
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Realizar la solicitud HTTP
response = requests.get(url, headers=headers)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    print(f"Accediendo a la página: {url}")
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Buscar los títulos de las noticias dentro de las etiquetas <h2> con el atributo data-testid="card-headline"
    articulos = soup.find_all('h2', {'data-testid': 'card-headline'})
    
    # Crear una lista para almacenar las noticias
    noticias = []

    # Extraer título y enlace de cada noticia
    for articulo in articulos:
        titulo = articulo.get_text(strip=True)
        enlace = articulo.find_parent('a')['href'] if articulo.find_parent('a') else None
        
        # Añadir la noticia a la lista
        if enlace:
            # Asegurar que el enlace sea completo
            if enlace.startswith('/'):
                enlace = f"https://www.bbc.com{enlace}"
            noticias.append({
                'titulo': titulo,
                'enlace': enlace
            })
    
    # Convertir los datos a un DataFrame de pandas
    df = pd.DataFrame(noticias)

    # Guardar los datos en un archivo JSON
    df.to_json('noticias_bbc.json', orient='records', lines=True, force_ascii=False)

    print(f"Datos guardados en 'noticias_bbc.json'")

else:
    print(f"Error al acceder a la página. Código de estado: {response.status_code}")

Accediendo a la página: https://www.bbc.com/news
Datos guardados en 'noticias_bbc.json'


In [96]:
# URL de la página principal de CNN
url = 'https://www.cnn.com/'

# Encabezados para evitar restricciones
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Realizar la solicitud HTTP
response = requests.get(url, headers=headers)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    print(f"Accediendo a la página: {url}")
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Buscar los títulos de las noticias dentro de las etiquetas <h2> con la clase específica
    articulos = soup.find_all('h2', class_='container__title_url-text container_lead-package__title_url-text')
    
    # Crear una lista para almacenar las noticias
    noticias = []

    # Extraer título y enlace de cada noticia
    for articulo in articulos:
        titulo = articulo.get_text(strip=True)
        enlace = articulo.find_parent('a')['href'] if articulo.find_parent('a') else None
        
        # Añadir la noticia a la lista
        if enlace:
            # Asegurar que el enlace sea completo
            if enlace.startswith('/'):
                enlace = f"https://www.cnn.com{enlace}"
            noticias.append({
                'titulo': titulo,
                'enlace': enlace
            })
    
    # Convertir los datos a un DataFrame de pandas
    df = pd.DataFrame(noticias)

    # Guardar los datos en un archivo JSON
    df.to_json('noticias_cnn.json', orient='records', lines=True, force_ascii=False)

    print(f"Datos guardados en 'noticias_cnn.json'")

else:
    print(f"Error al acceder a la página. Código de estado: {response.status_code}")

Accediendo a la página: https://www.cnn.com/
Datos guardados en 'noticias_cnn.json'


In [97]:
# URL de la página principal de The Guardian
url = 'https://www.theguardian.com/'

# Encabezados para evitar restricciones
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Realizar la solicitud HTTP
response = requests.get(url, headers=headers)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    print(f"Accediendo a la página: {url}")
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Buscar los títulos de las noticias dentro de las etiquetas <a> con el atributo 'data-link-name'
    articulos = soup.find_all('a', attrs={'data-link-name': lambda x: x and 'card' in x})
    
    # Crear una lista para almacenar las noticias
    noticias = []

    # Extraer título y enlace de cada noticia
    for articulo in articulos:
        titulo = articulo.get('aria-label', 'Sin título').strip()
        enlace = articulo['href'] if articulo.has_attr('href') else None
        
        # Añadir la noticia a la lista
        if enlace:
            # Asegurar que el enlace sea completo
            if enlace.startswith('/'):
                enlace = f"https://www.theguardian.com{enlace}"
            noticias.append({
                'titulo': titulo,
                'enlace': enlace
            })
    
    # Convertir los datos a un DataFrame de pandas
    df = pd.DataFrame(noticias)

    # Guardar los datos en un archivo JSON
    df.to_json('noticias_the_guardian.json', orient='records', lines=True, force_ascii=False)

    print(f"Datos guardados en 'noticias_the_guardian.json'")

else:
    print(f"Error al acceder a la página. Código de estado: {response.status_code}")

Accediendo a la página: https://www.theguardian.com/
Datos guardados en 'noticias_the_guardian.json'


In [98]:
# URL de la página principal de Al Jazeera
url = 'https://www.aljazeera.com/'

# Encabezados para evitar restricciones
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Realizar la solicitud HTTP
response = requests.get(url, headers=headers)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    print(f"Accediendo a la página: {url}")
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Buscar los títulos de las noticias dentro de las etiquetas <h3> con la clase 'gc__title'
    articulos = soup.find_all('h3', class_='gc__title')
    
    # Crear una lista para almacenar las noticias
    noticias = []

    # Extraer título y enlace de cada noticia
    for articulo in articulos:
        enlace_element = articulo.find('a', class_='u-clickable-card__link')
        titulo = enlace_element.find('span').get_text(strip=True) if enlace_element else "Sin título"
        enlace = enlace_element['href'] if enlace_element and enlace_element.has_attr('href') else None
        
        # Añadir la noticia a la lista
        if enlace:
            # Asegurar que el enlace sea completo
            if enlace.startswith('/'):
                enlace = f"https://www.aljazeera.com{enlace}"
            noticias.append({
                'titulo': titulo,
                'enlace': enlace
            })
    
    # Convertir los datos a un DataFrame de pandas
    df = pd.DataFrame(noticias)

    # Guardar los datos en un archivo JSON
    df.to_json('noticias_aljazeera.json', orient='records', lines=True, force_ascii=False)

    print(f"Datos guardados en 'noticias_aljazeera.json'")

else:
    print(f"Error al acceder a la página. Código de estado: {response.status_code}")

Accediendo a la página: https://www.aljazeera.com/
Datos guardados en 'noticias_aljazeera.json'


In [ ]:
# Directorio donde están los archivos JSON
directorio_json = './json_files/'  # Cambia esta ruta si es necesario
directorio_csv = './csv_files/'  # Carpeta para guardar los CSV

# Asegurarse de que las carpetas existen
os.makedirs(directorio_json, exist_ok=True)
os.makedirs(directorio_csv, exist_ok=True)

# Procesar cada archivo JSON
for archivo in os.listdir(directorio_json):
    if archivo.endswith('.json'):
        # Cargar JSON como DataFrame
        ruta_json = os.path.join(directorio_json, archivo)
        df = pd.read_json(ruta_json, lines=True)
        
        # Guardar como CSV
        nombre_csv = archivo.replace('.json', '.csv')
        ruta_csv = os.path.join(directorio_csv, nombre_csv)
        df.to_csv(ruta_csv, index=False, encoding='utf-8')
        print(f"Convertido: {archivo} -> {nombre_csv}")

In [56]:
# Subir el archivo CSV a Couchbase 
try:
    with open('noticias_ecuador.csv', 'r', encoding='utf-8') as f:
        csv_content = f.read()
    
    cb_coll.insert('noticias_ecuador_csv', {'content': csv_content})
    print("CSV content inserted into Couchbase.")
except CouchbaseException as e:
    print(f"Error inserting CSV content into Couchbase: {e}")
except Exception as e:
    print(f"General error: {e}")

CSV content inserted into Couchbase.


In [3]:
# Lista para almacenar los DataFrames
csv_combinados = []

# Leer todos los CSV en el directorio
for archivo_csv in os.listdir(directorio_csv):
    if archivo_csv.endswith('.csv'):
        ruta_csv = os.path.join(directorio_csv, archivo_csv)
        df = pd.read_csv(ruta_csv)
        csv_combinados.append(df)

# Combinar todos los DataFrames en uno solo
df_combinado = pd.concat(csv_combinados, ignore_index=True)

# Guardar el archivo combinado como CSV
df_combinado.to_csv('noticias_ecuador.csv', index=False, encoding='utf-8')
print("Todos los CSV se han combinado en 'noticias_ecuador.csv'")


Todos los CSV se han combinado en 'noticias_ecuador.csv'


In [9]:
# Subir el archivo CSV a Couchbase 
try:
    with open('noticias_ecuador.csv', 'r', encoding='utf-8') as f:
        csv_content = f.read()
    
    cb_coll.insert('noticias_ecuador_csv', {'content': csv_content})
    print("CSV content inserted into Couchbase.")
except CouchbaseException as e:
    print(f"Error inserting CSV content into Couchbase: {e}")
except Exception as e:
    print(f"General error: {e}")

CSV content inserted into Couchbase.
